# Example of flux BC building topography, currently only 2 timesteps


In [ ]:
import matplotlib.pyplot as pyplot
import matplotlib.pylab as pylab
import numpy as np
import underworld as uw
import math
from underworld import function as fn
import glucifer
import os

Setup parameters
-----

Set simulation parameters. The box length is chosen for direct comparison to the benchmark values.

In [ ]:
resUnit = 40
boxLength = 2.0
boxHeight = 1.0
elType = "Q1/dQ0"
resx = int(boxLength*resUnit)
resy = int(boxHeight*resUnit)
minCoord = [0.,0.]
maxCoord = [boxLength,boxHeight]
eta = 1.0

In [ ]:
mesh = uw.mesh.FeMesh_Cartesian( elementType = (elType), 
                                 elementRes  = (resx, resy), 
                                 minCoord    = minCoord, 
                                 maxCoord    = maxCoord )

velocityField = uw.mesh.MeshVariable( mesh=mesh,         nodeDofCount=2 )
stressField   = uw.mesh.MeshVariable( mesh=mesh,         nodeDofCount=3 )
pressureField = uw.mesh.MeshVariable( mesh=mesh.subMesh, nodeDofCount=1 )

In [ ]:
velocityField.data[:] = [0.,0.]
pressureField.data[:] = 0.
stressField.data[:] = [0.0,0.0,0.0]

for ii in mesh.specialSets['MinJ_VertexSet']:
    coord = mesh.data[ii]
    stressField.data[ii] = [0.0,-0.6-0.15*np.exp(-5*(coord[0]-1)**2.0), 0.0]

In [ ]:
# Initialise a swarm.
swarm = uw.swarm.Swarm( mesh=mesh, particleEscape=True )
advector= uw.systems.SwarmAdvector(velocityField, swarm, order=2)

# Add a data variable which will store an index to determine material.
materialVariable = swarm.add_variable( dataType="double", count=1 )

# Create a layout object that will populate the swarm across the whole domain.
swarmLayout = uw.swarm.layouts.GlobalSpaceFillerLayout( swarm=swarm, particlesPerCell=20 )
# Populate.
swarm.populate_using_layout( layout=swarmLayout )

# material 0 - compressible Lambda=10, density = 0
# material 1 - incompressible Lambda=0, density = 1

materialVariable.data[:]=0
for index,coord in enumerate(swarm.particleCoordinates.data):
    if coord[1] < boxHeight*0.6:
        materialVariable.data[index]=1

        
population_control = uw.swarm.PopulationControl(swarm, 
                                                aggressive=True,splitThreshold=0.15, maxDeletions=2,maxSplits=10,
                                                particlesPerCell=20)

# build tracer swarm for fluid level
tracerSwarm = uw.swarm.Swarm( mesh=mesh )
tracerAdvector= uw.systems.SwarmAdvector(velocityField, tracerSwarm, order=2)
particleCoordinates = np.zeros((10,2))
particleCoordinates[:,0] = np.linspace(0,2,10)
particleCoordinates[:,1] = 0.6
tracerSwarm.add_particles_with_coordinates(particleCoordinates)

In [ ]:
# Lambda = materialVariable * fn.misc.constant(10.0) -  is created for pseudo compressibility
Lambda = uw.function.branching.conditional( [ (materialVariable < 0.1, 10.0),
                                              (True,             0.0) ] )
fn_force = materialVariable * (0.0,-1.0)

In [ ]:
iWalls = mesh.specialSets["MinI_VertexSet"] + mesh.specialSets["MaxI_VertexSet"]
jWalls = mesh.specialSets["MinJ_VertexSet"] + mesh.specialSets["MaxJ_VertexSet"]
bottomWall = mesh.specialSets["MinJ_VertexSet"]
allWalls = iWalls + jWalls

# Now, using these sets, decide which degrees of freedom (on each node) should be considered Dirichlet.
stokesBC = uw.conditions.DirichletCondition( variable      = velocityField, 
                                             indexSetsPerDof = (iWalls, jWalls-bottomWall) )

# add neumann bcs
nbc      = uw.conditions.NeumannCondition( flux=stressField, variable = velocityField, 
                                             nodeIndexSet = bottomWall )

In [ ]:
fig1 = glucifer.Figure()
fig1.append( glucifer.objects.Points(swarm, materialVariable, fn_size=2.0 ) )
fig1.append( glucifer.objects.Mesh(mesh) )

fig1.show()

fig2 = glucifer.Figure()
fig2.append( glucifer.objects.VectorArrows(mesh, velocityField, scaling=8.0e2 ) )
fig2.append( glucifer.objects.Surface(mesh, pressureField) )

**Create a Stokes system**

In [ ]:
stokesPIC = uw.systems.Stokes( velocityField = velocityField, 
                               pressureField = pressureField,
                               conditions    = [stokesBC, nbc],
                               fn_viscosity  = 1.0e6, 
                               fn_bodyforce  = fn_force,
                               fn_lambda     = Lambda )
solver = uw.systems.Solver( stokesPIC )

In [ ]:
vdotv = fn.math.dot(velocityField,velocityField)
v2sum_integral  = uw.utils.Integral( mesh=mesh, fn=vdotv )
volume_integral = uw.utils.Integral( mesh=mesh, fn=1. )

In [ ]:
# general checkpoint function
def checkpoint( velocityField, pressureField, mesh, materialVariable, swarm, index, prefix=None ):
    
    if not hasattr( checkpoint, 'prefix'):
        if prefix is not None:
            import os
            if not os.path.exists('./'+prefix):
                try:
                    os.makedirs("./"+prefix+'/')
                except:
                    raise
        else:
            prefix=' '
        checkpoint.prefix=prefix
    
    
    prefix = checkpoint.prefix
    
    ii = str(index)
    
    # see if we have already saved the mesh. It only needs to be saved once
    if not hasattr( checkpoint, 'mH' ):
        checkpoint.mH = mesh.save(prefix+"mesh.h5")

    mh = checkpoint.mH
    vH = velocityField.save(prefix+"velocity"+ii+".h5")
    pH = pressureField.save(prefix+"pressure"+ii+".h5")
    sH = swarm.save(prefix+"swarm"+ii+".h5")
    smH = materialVariable.save(prefix+"matVar"+ii+".h5")
    trH = tracerSwarm.save(prefix+'tracers'+ii+'.h5')

    velocityField.xdmf(prefix+"velocity"+ii, vH, 'velocity', mh, 'mesh')
    pressureField.xdmf(prefix+"pressure"+ii, pH, 'pressure', mh, 'mesh')
    materialVariable.xdmf(prefix+"swarm"+ii, smH, 'materials', sH, 'swarm')

In [ ]:
steps         = 0
finalTimestep = 2

In [ ]:
# prefix='rubbish/'
# if prefix is None:
#     prefix=''
# else:
#     try:
#         import os
#         if not os.path.exists("./"+prefix+"/"):
#             os.makedirs("./"+prefix+'/')
#     except:
#         raise

# outfile = open(prefix+'buildMount.txt', 'w+')
# outfile.write( "Steps, Timestep, vrms, meanHeight\n")
# print "Steps, Timestep, Volume\n"
# lWallx = mesh.minCoord[0]
# rWallx = mesh.maxCoord[0]
# fig1.save_image(prefix+'particle.'+str(steps)+'.png')
# fig2.save_image(prefix+'buildMount.'+str(steps)+'.png')
topology = tracerSwarm.particleCoordinates.data[:,1].copy()
# print topology
# deltaTopo = 1.0
while steps<finalTimestep:
    # Get solution for initial configuration.
    solver.solve()

    # Advect using this timestep size.
    dt = advector.get_max_dt()
        
    advector.integrate(dt)  
    tracerAdvector.integrate(dt)


    newtopology = tracerSwarm.particleCoordinates.data[:,1]
    deltaTopo   = np.linalg.norm(newtopology-topology)/np.linalg.norm(topology)
    topology    = newtopology.copy()
    
#     with mesh.deform_mesh():
#         for index, coord in enumerate(mesh.data):
#             mesh.data[index][0] =+ dt * velocityField.data[index][0]
    
    population_control.repopulate()   
    
    steps += 1
#     print "swarm number {}, steps {}, deltaTopo {}".format(swarm.particleGlobalCount,steps, deltaTopo)
    
#     if ((steps % 2) == 0):
#         checkpoint(velocityField=velocityField, pressureField=pressureField, mesh=mesh, 
#                    materialVariable=materialVariable, swarm=swarm, index=steps, prefix=prefix)
        
#     fig1.save_image(prefix+'particle.'+str(steps)+'.png')
#     fig2.save_image(prefix+'buildMount.'+str(steps)+'.png')
    
    v2int = v2sum_integral.evaluate()[0]
    vol = volume_integral.evaluate()[0]
#     string = "{0}, {1:.3e}, {2:.3e}, {3:.3e}\n".format(steps,dt,np.sqrt(v2int/vol), tracerSwarm.particleCoordinates.data[0][1])
#     print string+"\n"
#     outfile.write(string)

# outfile.close()